In [2]:
pip install torch torch-geometric wandb pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
import torch.nn.functional as F
import glob
import numpy as np
import pandas as pd
import wandb
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

from torch.utils.data import Dataset, DataLoader


In [3]:
# Initialize Weights & Biases
wandb.init(project="rocket-league-gnn")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: boysle (boysle-boun). Use `wandb login --relogin` to force relogin


In [40]:
class RocketLeagueDataset(Dataset):
    def __init__(self, root_dir):
        super().__init__()
        self.file_paths = glob.glob(os.path.join(root_dir, "**", "*.csv"), recursive=True)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        return self.process_file(file_path)

    def process_file(self, file_path):
        df = pd.read_csv(file_path)
        data_list = []

        for _, row in df.iterrows():
            data_list.append(self.process_row(row, mirror=False))  # Original
            data_list.append(self.process_row(row, mirror=True))   # Mirrored

        return data_list  # Return only one file’s data at a time

    def process_row(self, row, mirror=False):
        # Extract player features (nodes)
        player_features = []
        for i in range(6):  # 6 players
            start_idx = i * 9
            features = row[start_idx:start_idx + 9].values.astype('float16')  # Convert to float16
            if mirror:
                features[0] *= -1  # Mirror x-axis
                features[1] *= -1  # Mirror y-axis
                features[3] *= -1  # Mirror x velocity
                features[4] *= -1  # Mirror y velocity
            player_features.append(features)
        x = torch.tensor(np.array(player_features), dtype=torch.float16)  # Convert the list to a NumPy array first

        # Ball state vector
        state_vector = torch.tensor([
            row['ball_pos_x'], row['ball_pos_y'], row['ball_pos_z']], dtype=torch.float16)

        if mirror:
            state_vector[0] *= -1  # Mirror ball x-axis
            state_vector[1] *= -1  # Mirror ball y-axis

        # Labels
        y = torch.tensor([row['team_0_goal_prev_5s'], row['team_1_goal_prev_5s']], dtype=torch.float16)

        # Edge index (fully connected graph for 6 players)
        edge_index = torch.combinations(torch.arange(6), r=2).t().contiguous()

        return Data(x=x, edge_index=edge_index, state=state_vector, y=y)

# Create a DataLoader that loads one file at a time
dataset_root = "E:\\Raw RL Esports Replays\\Day 3 Swiss Stage"
dataset = RocketLeagueDataset(dataset_root)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=lambda x: torch.utils.data.dataloader.default_collate(x))

In [42]:
# Define GCN model
class RocketLeagueGCN(torch.nn.Module):
    def __init__(self, input_dim, state_dim, hidden_dim, output_dim):
        super().__init__()
        self.gcn1 = GCNConv(input_dim, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim + state_dim, output_dim)

    def forward(self, data):
        x, edge_index, state = data.x, data.edge_index, data.state
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))
        x = torch.cat([x.mean(dim=0), state], dim=-1)  # Aggregate graph + state vector
        return torch.sigmoid(self.fc(x))  # Output probability

In [44]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RocketLeagueGCN(input_dim=6, state_dim=3, hidden_dim=32, output_dim=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.BCELoss()

In [46]:
from torch_geometric.loader import DataLoader  # Correct import

# Create a DataLoader that loads batches correctly
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=Batch.from_data_list)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:  # ✅ Now batch is a Batch object
        batch = batch.to(device)  # ✅ No more AttributeError!
        optimizer.zero_grad()
        out = model(batch)
        loss = loss_fn(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")


AttributeError: 'list' object has no attribute 'to'

In [ ]:
# Save model
torch.save(model.state_dict(), "rocket_league_gcn.pth")
wandb.save("rocket_league_gcn.pth")